# Importing Dependencies

In [ ]:
!pip install mediapipe opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 9.2 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import numpy as np


## Detections

In [ ]:
cap = cv2.VideoCapture(0)
mp_drawing = mp.solutions.drawing_utils #to draw the landmarks points
mp_pose = mp.solutions.pose #to iniialise mediapipe

In [ ]:
#call pose method and loop over frames in the video
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor: default OpenCV capture is in Blue Green Red format.
        #hence convert to Red Green Blue fror further operations
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #memory saving

        # make detection
        results = pose.process(image)

        # Recolor back to BGR as rendering in opencv requires that format
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections and specify the joint and connection properties respectively using mp_drawing
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow("Feed",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Sample Landmarks

In [ ]:
#coordinates of the landmoarks
results.pose_landmarks

NameError: ignored

## Determining Joints and Mapping

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow("Feed",image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

NameError: ignored

In [ ]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

In [ ]:
len(landmarks)

# Calculate Angles

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #mid
    c = np.array(c) #end

    #tan inverse and xy coordinates to find angle in radians
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    #convert to degree
    angle = np.abs(radians*180.0/np.pi)

    #angles to be kept less than 180
    if angle >180.0:
        angle = 360-angle

    return angle

In [ ]:
#get coordinates
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y] #11
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y] #13
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] #15
hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]  #23
knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y] #25
ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y] #27

In [ ]:
shoulder, elbow, wrist

In [ ]:
calculate_angle(shoulder, elbow, wrist)

In [ ]:
#View angle values: multiply normalised values to dimensions of video, cv2 needs tuple and int
tuple(np.multiply(elbow, [640, 480]).astype(int))

## Donkey Kick Counter

### Explanation of logic: <br><br>
Base conditions <br>
Upper body: <br>
shoulder-elbow-wrist should form 180 (straight): 11 13 15 <br>
hip-shoulder-elbow should form 90 (perpendicular): 23 11 13 <br>

Lower body: <br>
hip-knee-ankle should form 90 (perpendicular): 23 25 27 <br><br>

Stages <br>
Down: shoulder-hip-knee form 90: 11 23 25 <br>
Up: shoudler-hip-knee form 180: 11 23 25 <br><br>

![Screenshot%202023-04-13%20003416.png](attachment:Screenshot%202023-04-13%20003416.png)

In [ ]:
cap = cv2.VideoCapture(0)

# donkey kick counter variables
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y] #11
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y] #13
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] #15
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]  #23
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y] #25
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y] #27

            # calculate all required angles
            angle1 = calculate_angle(shoulder, elbow, wrist)
            angle2 = calculate_angle(hip, shoulder, elbow)
            angle3 = calculate_angle(hip, knee, ankle)
            angle4 = calculate_angle(shoulder, hip, knee)

            # to view all angles
            cv2.putText(image, str(angle1),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(angle2),
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(angle3),
                           tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(angle4),
                           tuple(np.multiply(hip, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # counter logic implementation
            while (angle1 > 170 and angle1 < 190) and (angle2 > 70 and angle2 < 100) and (angle3 > 70 and angle3 < 100):
                if angle3 >70 and angle3 < 100:
                    stage = "down"
                if angle3 > 170 and angle3 < 190 and stage=='down':
                    stage = "up"
                    counter+=1
                    print(counter)

        except:
            pass

        # display counter box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # counter data
        cv2.putText(image, 'DONKEY KICKS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, '', (65,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Output', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

## Weight Lift Counter

### Explanation of Logic <br><br>
Stages <br>
Down: shoulder-elbow-wrist: 11 13 15 and hip-shoulder-elbow : 23 11 13 should each be close to 0<br>
Up: shoulder-elbow-wrist: 11 13 15 and hip-shoulder-elbow : 23 11 13 should each be close to 0<br><br>
![Screenshot%202023-04-13%20010136.png](attachment:Screenshot%202023-04-13%20010136.png)


In [ ]:
cap = cv2.VideoCapture(0)

# lift variables
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y] #11
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y] #13
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] #15
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]  #23
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y] #25
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y] #27

            # calculate all required angles
            angle1 = calculate_angle(shoulder, elbow, wrist)
            angle2 = calculate_angle(hip, shoulder, elbow)

            # to view all angles
            cv2.putText(image, str(angle1),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )
            cv2.putText(image, str(angle2),
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # counter logic implementation
            if (angle1 > 0 and angle1 < 10) and (angle2 > 0 and angle2 < 10):
                stage = "down"
            if (angle1 > 170 and angle1 < 190) and (angle2 > 170 and angle2 < 190) and stage=='down':
                stage = "up"
                counter+=1
                print(counter)

        except:
            pass

        # display counter box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # counter data
        cv2.putText(image, 'LIFTS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Output', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()